<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale and Batch Processing:<br>Apache Spark on Cloud Pak for Data with IBM Analytics Engine

This notebook must be run in the Python 3.10 runtime environment. It requires Watson OpenScale service credentials.

The notebook configures Watson OpenScale to monitor the German Credit Risk model. Use the notebook to enable quality, drift, fairness and explainability monitoring and run on-demand evaluations. Before you can run the notebook, you must have the following resources:

1. The common configuration JSON, Drift Configuration and Explainability Configuration archives generated by using the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb).
2. Feedback, payload, drifted transactions, explanations queue and result tables in an IBM DB2 storage that use the data description language statements (DDLs) that are generated as part of running the previous common configuration notebook.

In [1]:
# ----------------------------------------------------------------------------------------------------
# IBM Confidential
# OCO Source Materials
# 5737-H76
# Copyright IBM Corp. 2021-2023
# The source code for this Notebook is not published or other-wise divested of its trade
# secrets, irrespective of what has been deposited with the U.S.Copyright Office.
# ----------------------------------------------------------------------------------------------------

VERSION = "iae-jdbc-1.0"

# Version History
# iae-jdbc-1.0 : Add partitioning information
# <no-verion>  : Initial release

## Contents

1. [Setup](#setup)
2. [Configure Watson OpenScale](#openscale)
3. [Set up a subscription](#subscription)
4. [Quality monitoring](#quality)
5. [Drift monitoring](#drift)
6. [Fairness monitoring](#fairness)
7. [Explainability monitoring](#explainability)

# 1. Setup <a name="setup"></a>

## Package installation

First import some of the packages you need to use. After you finish installing the following software packages, restart the kernel.

In [2]:
import warnings
warnings.filterwarnings("ignore")
%env PIP_DISABLE_PIP_VERSION_CHECK=1

env: PIP_DISABLE_PIP_VERSION_CHECK=1


In [ ]:
!pip install -U ibm-watson-openscale --no-cache | tail -n 1

In [3]:
!pip show ibm-watson-openscale

Name: ibm-watson-openscale
Version: 3.0.39
Summary: Client library for IBM Watson OpenScale
Home-page: https://github.ibm.com/watson-developer-cloud/openscale-python-sdk
Author: IBM Watson OpenScale
Author-email: kishore.patel@in.ibm.com
License: Apache 2.0
Location: /opt/conda/envs/Python-RT24.1-Premium/lib/python3.11/site-packages
Requires: ibm-cloud-sdk-core, pandas, python-dateutil, requests
Required-by: 


## Configure credentials

Provide your IBM Watson OpenScale credentials in the following cell:

In [4]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": "",
    "instance_id": ""
}

## Specify model details

### Service provider and subscription metadata

In [5]:
# Service Provider

SERVICE_PROVIDER_NAME = ""
SERVICE_PROVIDER_DESCRIPTION = ""

# Subscription

SUBSCRIPTION_NAME = ""
SUBSCRIPTION_DESCRIPTION = ""

### IBM Analytics Engine - Spark

Make sure that the Apache Spark manager on IBM Analytics Engine is running, and then provide the following details:

- IAE_SPARK_DISPLAY_NAME: _Display Name of the Spark instance in IBM Analytics Engine_
- IAE_SPARK_JOBS_ENDPOINT: _Spark Jobs Endpoint for IBM Analytics Engine_
- IBM_CPD_VOLUME: _IBM Cloud Pak for Data storage volume name_
- IBM_CPD_USERNAME: _IBM Cloud Pak for Data username_
- IBM_CPD_APIKEY: _IBM Cloud Pak for Data API key_
- IAE_SPARK_DESCRIPTION: _Custom description for the Spark instance_

In [6]:
IAE_SPARK_DISPLAY_NAME = ""
IAE_SPARK_JOBS_ENDPOINT = ""
IBM_CPD_VOLUME = ""
IBM_CPD_USERNAME = ""
IBM_CPD_APIKEY = ""
IAE_SPARK_NAME = ""
IAE_SPARK_DESCRIPTION = ""

#### Provide Spark Resource Settings

To configure how much of your Spark Cluster resources this job can consume, edit the following values:

- max_num_executors: _Maximum Number of executors to launch for this session_
- min_executors: _Minimum Number of executors to launch for this session_
- executor_cores: _Number of cores to use for each executor_   
- executor_memory: _Amount of memory (in GBs) to use per executor process_
- driver_cores: _Number of cores to use for the driver process_
- driver_memory: _Amount of memory (in GBs) to use for the driver process_

In [7]:
spark_parameters = {
    "max_num_executors": 2,
    "min_num_executors": 1,
    "executor_cores": 3,
    "executor_memory": 2,
    "driver_cores": 2,
    "driver_memory": 2
}

### Storage Inputs

Please enter a name and description for your JDBC Storage

- JDBC_CONNECTION_NAME: _Custom display name for the JDBC Storage Connection_
- JDBC_CONNECTION_DESCRIPTION: _Custom description for the JDBC Storage Connection_

To connect to your JDBC storage, you must provide the following details:

 - JDBC_HOST: Hostname of the JDBC Connection
 - JDBC_PORT: Port of the JDBC Connection
 - JDBC_USE_SSL: Boolean Flag to indicate whether to use SSL while connecting.
 - JDBC_SSL_CERTIFICATE: SSL Certificate [Base64 encoded string] of the JDBC Connection. Ignored if JDBC_USE_SSL is False.
 - JDBC_DRIVER: Class name of the JDBC driver to use to connect.
 - JDBC_USERNAME: Username of the JDBC Connection
 - JDBC_PASSWORD: Password of the JDBC Connection
 - JDBC_DATABASE_NAME: Name of the Database to connect to.

In [8]:
JDBC_CONNECTION_NAME = ""
JDBC_CONNECTION_DESCRIPTION = ""

JDBC_HOST = ""
JDBC_PORT = ""
JDBC_USE_SSL = ""
JDBC_SSL_CERTIFICATE = ""
JDBC_DRIVER = ""
JDBC_USERNAME = ""
JDBC_PASSWORD = ""
JDBC_DATABASE_NAME = ""

In [9]:
num_partitions_recommended = 12

if spark_parameters:
    executors = spark_parameters.get("max_num_executors", 2)
    cores = spark_parameters.get("executor_cores", 2)
    num_partitions_recommended = 2 * executors * cores
    
print("{} is the recommended value for number of partitions in your data. Please change this value as per your data.".format(num_partitions_recommended))

12 is the recommended value for number of partitions in your data. Please change this value as per your data.


- **PARTITION_COLUMN**: The column to help Spark read and write data using multiple workers in your JDBC storage. This will help improve the performance of your Spark jobs. The default value is set to `wos_partition_column`.
- **NUM_PARTITIONS**: The maximum number of partitions that Spark can divide the data into. In JDBC, it also means the maximum number of connections that Spark can make to the JDBC store for reading/writing data. 

The recommended value is calculated in the above cell as a multiple of total workers allotted for this job. You can use the same value or change it in the next cell.

In [10]:
PARTITION_COLUMN = "wos_partition_column"
NUM_PARTITIONS = num_partitions_recommended

### Feedback table metadata

The quality monitor stores metadata in the feedback table. To configure the quality monitor, you must provide the following details. To skip quality monitoring, run the following cell to initialize variables with the value of `None`.

- FEEDBACK_SCHEMA_NAME: _Schema name where feedback table is present_
- FEEDBACK_TABLE_NAME: _Name of the feedback table_

In [11]:
#feedback

FEEDBACK_SCHEMA_NAME = None
FEEDBACK_TABLE_NAME = None

### Payload and drift table metadata

The drift monitor stores metadata in the payload and drift tables. To configure the drift monitor, you must provide the following details. To skip drift monitoring, run the following cell to initialize variables with the value of `None`.

- PAYLOAD_SCHEMA_NAME: _Schema name where payload logging table is present_
- PAYLOAD_TABLE_NAME: _Name of the payload logging table_
- DRIFT_SCHEMA_NAME: _Schema name where drifted transactions table is present_
- DRIFT_TABLE_NAME: _Name of the drifted transactions table_

In [12]:
#payload logging

PAYLOAD_SCHEMA_NAME = None
PAYLOAD_TABLE_NAME = None

#drift

DRIFT_SCHEMA_NAME = None
DRIFT_TABLE_NAME = None

### Explainability table metadata

The explainability monitor requires the queue and result tables. The payload table can also be used as the queue table. To configure the explainability monitor, you must provide the following details. To skip explainability monitoring, run the following cell to initialize variables with the value of `None`.

- EXPLAINABILITY_SCHEMA_NAME: _Schema name where explanations queue, result tables are present_
- EXPLAINABILITY_QUEUE_TABLE_NAME: _Name of the explanations queue table_
- EXPLAINABILITY_RESULT_TABLE_NAME: _Name of the explanations result table_

In [13]:
#explainability

EXPLAINABILITY_SCHEMA_NAME = None
EXPLAINABILITY_QUEUE_TABLE_NAME = None
EXPLAINABILITY_RESULT_TABLE_NAME = None

# 2. Configure Watson OpenScale <a name="openscale"></a>

### Import the required libraries and set up the Watson OpenScale client

In [14]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator
from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import *

authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS["url"],
        username=WOS_CREDENTIALS["username"],
        password=WOS_CREDENTIALS["password"],
        disable_ssl_verification=True
    )

wos_client = APIClient(authenticator=authenticator, service_url=WOS_CREDENTIALS["url"], service_instance_id=WOS_CREDENTIALS["instance_id"])

### Display Watson OpenScale datamart details

In [15]:
wos_client.data_marts.show()
data_marts = wos_client.data_marts.list().result.data_marts
data_mart_id = WOS_CREDENTIALS["instance_id"]

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


### Create a service provider

In [16]:
# Delete existing service provider with the same name as provided

service_providers = wos_client.service_providers.list().result.service_providers
for provider in service_providers:
    if provider.entity.name == SERVICE_PROVIDER_NAME:
        wos_client.service_providers.delete(service_provider_id=provider.metadata.id)
        break

In [17]:
# Add Service Provider

added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.CUSTOM_MACHINE_LEARNING,
        credentials={},
        operational_space_id="production",
        background_mode=False
    ).result

service_provider_id = added_service_provider_result.metadata.id

wos_client.service_providers.show()




 Waiting for end of adding service provider d90c6bf2-49c6-4179-9876-8b85b0247d95 






active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




None,active,WML_IAE4,custom_machine_learning,2024-07-02 07:00:41.816000+00:00,d90c6bf2-49c6-4179-9876-8b85b0247d95
None,active,RC - OpenScale Headless Service Provider,custom_machine_learning,2024-07-02 06:25:36.094000+00:00,ec2a2f93-18b0-4e35-a6ff-ca06cb96e6a5
99999999-9999-9999-9999-999999999999,active,Image Multiclass Watson Machine Learning V2_test,watson_machine_learning,2024-07-01 17:17:14.696000+00:00,a7ca157a-de07-457a-8c4c-b1a2e998699c
99999999-9999-9999-9999-999999999999,active,Image Binary WML V2_test,watson_machine_learning,2024-07-01 16:48:33.812000+00:00,d9733409-7fdc-491b-8c6b-909e92f16bd6
None,active,WML_remote_spark_jdbc,custom_machine_learning,2024-07-01 09:40:35.190000+00:00,26b2af15-e396-4295-81d2-6912ab93912b
00000000-0000-0000-0000-000000000000,active,WML_IAE3,watson_machine_learning,2024-06-30 11:22:00.507000+00:00,d7ad1164-a387-462d-b495-dd25d6241404
None,active,OpenScale Headless Service Provider,custom_machine_learning,2024-06-29 03:21:50.385000+00:00,e67332c4-0f88-4a8f-9c41-396d28c07448
None,active,IAE3,custom_machine_learning,2024-06-28 05:11:53.191000+00:00,8ac485c9-66c8-4a9c-9786-57d2940f26e8
None,active,WML_IAE2,custom_machine_learning,2024-06-27 13:36:27.009000+00:00,8b65752d-0dc9-4715-a2a2-96ee19fb7ded
99999999-9999-9999-9999-999999999999,active,shreya-space,watson_machine_learning,2024-06-26 04:34:04.685000+00:00,34668a1a-ee92-4328-8361-05ba1c3f72bf


Note: First 10 records were displayed.


In [18]:
service_provide_details = wos_client.service_providers.get(service_provider_id=service_provider_id).result
print(service_provide_details)

{
  "metadata": {
    "id": "d90c6bf2-49c6-4179-9876-8b85b0247d95",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:service_provider:d90c6bf2-49c6-4179-9876-8b85b0247d95",
    "url": "/v2/service_providers/d90c6bf2-49c6-4179-9876-8b85b0247d95",
    "created_at": "2024-07-02T07:00:41.816000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAE4",
    "service_type": "custom_machine_learning",
    "credentials": {
      "secret_id": "2b03bac3-8f34-49aa-8aad-2639532867e9"
    },
    "operational_space_id": "production",
    "status": {
      "state": "active"
    }
  }
}


### Create integrated systems for Spark Engine and JDBC Storage

In [19]:
# Delete existing spark and jdbc integrated systems if present

integrated_systems = IntegratedSystems(wos_client).list().result.integrated_systems

for system in integrated_systems:
    if system.entity.name in (IAE_SPARK_NAME, JDBC_CONNECTION_NAME):
        print("Deleting integrated system {}".format(system.entity.name))
        IntegratedSystems(wos_client).delete(integrated_system_id=system.metadata.id)

Deleting integrated system jdbc for batch parity


#### Spark Engine

In [20]:
spark_engine_details = IntegratedSystems(wos_client).add(
    name=IAE_SPARK_NAME,
    description=IAE_SPARK_DESCRIPTION,
    type="spark",
    credentials={
        "username": IBM_CPD_USERNAME,
        "apikey": IBM_CPD_APIKEY
    },
    connection={
        "display_name": IAE_SPARK_DISPLAY_NAME,
        "endpoint": IAE_SPARK_JOBS_ENDPOINT,
        "volume": IBM_CPD_VOLUME,
        "location_type": "cpd_iae"
    }
).result

spark_engine_id = spark_engine_details.metadata.id
print(spark_engine_details)

{
  "metadata": {
    "id": "a7542e2c-f256-4228-bfc9-b42f27f3a400",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:a7542e2c-f256-4228-bfc9-b42f27f3a400",
    "url": "/v2/integrated_systems/a7542e2c-f256-4228-bfc9-b42f27f3a400",
    "created_at": "2024-07-02T07:00:47.437000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "WML_IAEKB_Spark",
    "type": "spark",
    "description": "WML_IAEKB_Spark",
    "credentials": {
      "secret_id": "c6ff2508-9182-47e5-be62-4e54b4f487fa"
    },
    "connection": {
      "display_name": "IAEBatchSpark",
      "endpoint": "https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/v4/analytics_engines/7d2a4875-563e-42d0-994f-518ebf6e1e42/spark_applications",
      "location_type": "cpd_iae",
      "volume": "cpd-instance::IAEBatchTest"
    }
  }
}


#### JDBC Storage

In [21]:
jdbc_url = "jdbc:db2://{}:{}/{}".format(JDBC_HOST, JDBC_PORT, JDBC_DATABASE_NAME)

jdbc_connection_details = IntegratedSystems(wos_client).add(
    name=JDBC_CONNECTION_NAME,
    description=JDBC_CONNECTION_DESCRIPTION,
    type="jdbc",
    credentials={
        "username": JDBC_USERNAME,
        "password": JDBC_PASSWORD,
        "jdbc_url": jdbc_url
    },
    connection={
        "location_type": "jdbc",
        "db_driver": JDBC_DRIVER,
        "use_ssl": JDBC_USE_SSL,
        "certificate": JDBC_SSL_CERTIFICATE,
    }
).result

jdbc_connection_id=jdbc_connection_details.metadata.id
print(jdbc_connection_details)

{
  "metadata": {
    "id": "5f31b2e2-e017-4bd8-a2d4-cda8ba410311",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:integrated_system:5f31b2e2-e017-4bd8-a2d4-cda8ba410311",
    "url": "/v2/integrated_systems/5f31b2e2-e017-4bd8-a2d4-cda8ba410311",
    "created_at": "2024-07-02T07:03:26.981000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "jdbc for batch parity",
    "type": "jdbc",
    "description": "jdbc connection from SDK",
    "credentials": {
      "secret_id": "09d89ae7-92f6-41f6-8e7a-a496d5ce7859"
    },
    "connection": {
      "certificate": "",
      "db_driver": "com.ibm.db2.jcc.DB2Driver",
      "location_type": "jdbc",
      "use_ssl": true
    }
  }
}


# 3. Set up a subscription<a name="subscription"></a>

In [22]:
# Delete an existing subscription with the provided name

subscriptions = wos_client.subscriptions.list().result.subscriptions
for sub in subscriptions:
    if sub.entity.deployment.name == SUBSCRIPTION_NAME:
        wos_client.subscriptions.delete(subscription_id=sub.metadata.id)
        break

# Display all subscriptions
wos_client.subscriptions.show()

405fe789-e294-42ce-9989-774d484205c3,model,WML_remote_spark_jdbc,00000000-0000-0000-0000-000000000000,ea8d064d-2eb9-48a0-97af-f385dd358843,WML_remote_spark_jdbc,26b2af15-e396-4295-81d2-6912ab93912b,active,2024-07-01 09:41:24.463000+00:00,3046b546-e422-4737-9123-a26e025d3cef
a4bb31e7-4c08-4697-8036-24e737b82c9c,model,[asset] RC - GCR Headless Subscription,00000000-0000-0000-0000-000000000000,9709c3ea-5eb8-4d97-babe-4c2ef72035a6,[asset] RC - GCR Headless Subscription,ec2a2f93-18b0-4e35-a6ff-ca06cb96e6a5,active,2024-07-02 06:25:59.135000+00:00,c4bcb314-aa1c-4ef0-a3a3-71d3f7993593
d1869f46-b9a0-4e99-bfbe-491074a4e40d,model,MNIST Model,00000000-0000-0000-0000-000000000000,211b2ebc-ec74-4090-a526-e896d5ed6166,MNIST Model deployment,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-01 17:17:44.331000+00:00,1b8ebb3c-e69a-4cc1-9a0a-57424546cab3
1fa2de0d-be8f-4808-a9e0-b3f472f2b7ff,model,Dog-Cat binary,00000000-0000-0000-0000-000000000000,4194c91a-0458-4afd-b8cc-a1f0a34ec077,Dog-Cat binary deployment,d9733409-7fdc-491b-8c6b-909e92f16bd6,active,2024-07-01 16:48:50.625000+00:00,8aafc9c3-bcbe-4655-b4c8-8ac57c8a74be
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-06-30 12:02:35.155000+00:00,7585a078-c1b6-4a77-9ab3-87be9fb6026c
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,error,2024-06-30 11:56:46.031000+00:00,3de14c6f-b8eb-4e61-af5f-17c83f388305
7b4ca85f-6a57-447c-8101-3a5e03e436e1,model,[asset] Multi-Output Regression Headless Subscription,00000000-0000-0000-0000-000000000000,7b4ca85f-6a57-447c-8101-3a5e03e436e1,deployment_[asset] Multi-Output Regression Headless Subscription,e67332c4-0f88-4a8f-9c41-396d28c07448,active,2024-06-29 03:22:11.360000+00:00,cb83c8bf-9e3b-42bf-9447-c4fb2706b56e
c89859aa-630e-4ac7-8394-b82139e06968,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,0c86658f-d40f-4c8e-9683-bf40808c8cc9,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,active,2024-06-28 05:22:29.196000+00:00,1f1c2f8b-99ef-4bbe-a2d3-06d098091fcb
94472b5e-6658-4169-8fb3-664ab3101efb,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,133d028a-826d-43e2-ac90-c040551f8f01,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,active,2024-06-28 05:17:56.016000+00:00,3bf32c6c-bdc2-4dc0-831c-df288edae02b
1a6421f0-94d7-466d-bcbe-14931bb4a7e9,model,My SDK Batch Subscription-db2,00000000-0000-0000-0000-000000000000,45490a50-f7e1-42e0-ac18-06cf8faae5af,My SDK Batch Subscription-db2,8ac485c9-66c8-4a9c-9786-57d2940f26e8,error,2024-06-28 05:12:09.112000+00:00,ca5936f2-9fd1-4876-9f0c-2b43ca5b8e7f


Note: First 10 records were displayed.


### Set subscription metadata

In the following cell, type a path to the common configuration JSON file that you created by running the [common configuration notebook](https://github.com/IBM/watson-openscale-samples/blob/main/Cloud%20Pak%20for%20Data/Batch%20Support/Configuration%20generation%20for%20OpenScale%20batch%20subscription.ipynb). After you edit the path information, run the cell to set the asset details and properties, the deployment details, the analytics engine details, and to add the required tables as data sources.

In [23]:
import uuid

common_configuration = None

with open("/path/to/dir/containing/common_config.json", "r") as fp:
    configuration_json = json.load(fp)
    common_configuration = configuration_json.get("common_configuration")
    if common_configuration is None:
        print("Please provide the correct path to the common configuration JSON")
    
# Set asset details
asset = Asset(
    asset_id=str(uuid.uuid4()),
    url="",
    name=SUBSCRIPTION_NAME,
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)

# Set deployment details
asset_deployment = AssetDeploymentRequest(
    deployment_id=str(uuid.uuid4()),
    name=SUBSCRIPTION_NAME,
    description=SUBSCRIPTION_DESCRIPTION,
    deployment_type="batch"
)

# Set asset properties 
asset_properties_request = AssetPropertiesRequest(
    label_column=common_configuration["label_column"],
    probability_fields=[common_configuration["probability"]],
    prediction_field=common_configuration["prediction"],
    feature_fields=common_configuration["feature_columns"],
    categorical_fields=common_configuration["categorical_columns"]
)

# Set analytics engine details
analytics_engine = AnalyticsEngine(
    type="spark",
    integrated_system_id=spark_engine_id,
    parameters = spark_parameters
)

# Add selected tables as data sources
data_sources = []
if FEEDBACK_SCHEMA_NAME is not None and FEEDBACK_TABLE_NAME is not None:
    feedback_data_source = DataSource(
        type="feedback",
        database_name=JDBC_DATABASE_NAME,
        schema_name=FEEDBACK_SCHEMA_NAME,
        table_name=FEEDBACK_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    data_sources.append(feedback_data_source)
    
if PAYLOAD_SCHEMA_NAME is not None and PAYLOAD_TABLE_NAME is not None \
    and DRIFT_SCHEMA_NAME is not None and DRIFT_TABLE_NAME is not None:
    payload_logging_data_source = DataSource(
        type="payload",
        database_name=JDBC_DATABASE_NAME,
        schema_name=PAYLOAD_SCHEMA_NAME,
        table_name=PAYLOAD_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    drifted_transactions_table_data_source = DataSource(
        type="drift",
        database_name=JDBC_DATABASE_NAME,
        schema_name=DRIFT_SCHEMA_NAME,
        table_name=DRIFT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_connection_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(payload_logging_data_source)
    data_sources.append(drifted_transactions_table_data_source)

if EXPLAINABILITY_DATABASE_NAME is not None and \
    EXPLAINABILITY_QUEUE_TABLE_NAME is not None and \
        EXPLAINABILITY_RESULT_TABLE_NAME is not None:

    explainability_queue_data_source = DataSource(
        type="explain_queue",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_QUEUE_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc", 
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={
            "partition_column": PARTITION_COLUMN,
            "num_partitions": NUM_PARTITIONS
        },
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_queue_data_source)

    explainability_result_data_source = DataSource(
        type="explain_result",
        database_name=EXPLAINABILITY_DATABASE_NAME,
        schema_name=EXPLAINABILITY_SCHEMA_NAME,
        table_name=EXPLAINABILITY_RESULT_TABLE_NAME,
        connection=DataSourceConnection(
            type="jdbc",
            integrated_system_id=jdbc_integrated_system_id
        ),
        parameters={},
        auto_create=True, #set it to False if table already exists
        status=DataSourceStatus(state="new")
    )
    
    data_sources.append(explainability_result_data_source)

In [24]:
# Adding the subscription

subscription_details = Subscriptions(wos_client).add(
    data_mart_id=data_mart_id,
    service_provider_id=service_provider_id,
    asset=asset,
    deployment=asset_deployment,
    asset_properties=asset_properties_request,
    analytics_engine=analytics_engine,
    data_sources=data_sources).result

subscription_id = subscription_details.metadata.id
print(subscription_details)

{
  "metadata": {
    "id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:e34b9b87-b6e1-4c53-b92e-cb80dea042be",
    "url": "/v2/subscriptions/e34b9b87-b6e1-4c53-b92e-cb80dea042be",
    "created_at": "2024-07-02T07:03:31.504000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "d90c6bf2-49c6-4179-9876-8b85b0247d95",
    "asset": {
      "asset_id": "438ca544-9bd1-48c2-8e8d-3de4ef4ca79b",
      "url": "",
      "name": "WML_IAE4",
      "asset_type": "model",
      "problem_type": "binary",
      "input_data_type": "structured"
    },
    "asset_properties": {
      "label_column": "Risk",
      "prediction_field": "prediction",
      "feature_fields": [
        "CheckingStatus",
        "LoanDuration",
        "CreditHistory",
        "LoanPurpose",
        "LoanAmount",
        "ExistingSavings",


In [25]:
# Check subscription status
import time

subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

07:16:01 preparing


07:16:16 preparing


07:16:31 preparing


07:16:46 active


In [26]:
# Add training, output, and input data schemas to the subscription

training_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/training_data_schema", value=common_configuration["training_data_schema"])
]

input_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/input_data_schema", value=common_configuration["input_data_schema"])
]

output_data_schema_patch_document=[
    JsonPatchOperation(op=OperationTypes.REPLACE, path="/asset_properties/output_data_schema", value=common_configuration["output_data_schema"])
]

wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=training_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=input_data_schema_patch_document)
wos_client.subscriptions.update(subscription_id=subscription_id, patch_document=output_data_schema_patch_document)

In [27]:
# Check subscription status
import time

subscription_status = None
while subscription_status not in ("active", "error"):
    subscription_status = wos_client.subscriptions.get(subscription_id).result.entity.status.state
    if subscription_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), subscription_status)
        time.sleep(15)
        
print(datetime.now().strftime("%H:%M:%S"), subscription_status)

07:17:12 active


# 4. Quality monitoring <a name="quality"></a>

### Enable the quality monitor

In the following code cell, default values are set for the quality monitor. You can change the default values by updating the optional `min_feedback_data_size` attribute in the `parameters` dict and set the quality threshold in the `thresholds` list.

In [28]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_feedback_data_size": 1000
}

thresholds = [{
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
}]

quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

quality_monitor_instance_id = quality_monitor_details.metadata.id
print(quality_monitor_details)

{
  "metadata": {
    "id": "9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "url": "/v2/monitor_instances/9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "created_at": "2024-07-02T07:17:18.072000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type": "relative",
        "delay_unit": "minute",
        "delay": 10
      },
      "rep

### Check monitor instance status

In [29]:
quality_status = None

while quality_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
    quality_status = monitor_instance_details.entity.status.state
    if quality_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_status)

07:17:23 active


In [30]:
monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "url": "/v2/monitor_instances/9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "created_at": "2024-07-02T07:17:18.072000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-02T07:17:18.564000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

### Run an on-demand evaluation

In [31]:
# Check Quality monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=quality_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "url": "/v2/monitor_instances/9200be2d-ce34-4ca0-b7ca-ef82721fb31b",
    "created_at": "2024-07-02T07:17:18.072000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-02T07:17:18.564000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "quality",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "min_feedback_data_size": 10
    },
    "thresholds": [
      {
        "metric_id": "area_under_roc",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "t

In [32]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "d839f091-1811-461c-866a-64be9ec2cab4",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:d839f091-1811-461c-866a-64be9ec2cab4",
    "url": "/v2/monitor_instances/9200be2d-ce34-4ca0-b7ca-ef82721fb31b/runs/d839f091-1811-461c-866a-64be9ec2cab4",
    "created_at": "2024-07-02T07:17:25.664000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "min_feedback_data_size": 10
    },
    "status": {
      "state": "running",
      "queued_at": "2024-07-02T07:17:25.658000Z",
      "started_at": "2024-07-02T07:17:25.664000Z",
      "operators": []
    }
  }
}


In [33]:
# Check run status

quality_run_status = None
while quality_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=quality_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    quality_run_status = monitoring_run_details.entity.status.state
    if quality_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), quality_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), quality_run_status)

07:17:26 running


07:17:56 running


07:18:26 running


07:18:56 running


07:19:26 running


...

07:20:56 running


07:21:26 running


07:21:56 running


07:22:27 running


07:22:57 finished


### Display quality metrics

In [34]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-07-02 07:22:28.532009+00:00,true_positive_rate,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.3333333333333333,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,area_under_roc,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.5952380952380952,0.8,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,precision,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.5,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,f1_measure,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.4,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,accuracy,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.7,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,log_loss,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.715401570634091,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,false_positive_rate,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.14285714285714285,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,area_under_pr,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.43333333333333335,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:22:28.532009+00:00,recall,7e3c4218-4e15-4048-85d0-43efe3e76a9f,0.3333333333333333,None,None,['model_type:original'],quality,9200be2d-ce34-4ca0-b7ca-ef82721fb31b,d839f091-1811-461c-866a-64be9ec2cab4,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be


# 5. Drift monitoring <a name="drift"></a>

### Enable the drift monitor

In the following code cell, type a path to the drift configuration tar ball.

In [35]:
wos_client.monitor_instances.upload_drift_model(
    model_path="drift_archive.tar.gz",
    data_mart_id=data_mart_id,
    subscription_id=subscription_id
).result

{'data_constraints': {'id': '81d2a92f-b601-45d9-bd09-134c1638e37b',
  'version': '0.02_batch',
  'columns': [{'name': 'Age',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CheckingStatus',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CreditHistory',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'CurrentResidenceDuration',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'Dependents',
    'dtype': 'numeric_discrete',
    'count': 50,
    'approx_count_distinct': 0,
    'sparse': False,
    'skip_learning': False},
   {'name': 'EmploymentDuration',
    'dtype': 'categorical',
    'count': 50,
    'approx_count_dis

In the following code cell, default values are set for the drift monitor. You can change the default values by updating the values in the `parameters` section. The `min_samples` parameter controls the number of records that triggers the drift monitor to run. The `drift_threshold` parameter sets the threshold in decimal format for the drift percentage to trigger an alert. The `train_drift_model` parameter controls whether to re-train the model based on the drift analysis.

In [36]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    "min_samples": 1000,
    "drift_threshold": 0.05,
    "train_drift_model": False
}

drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id
print(drift_monitor_details)

{
  "metadata": {
    "id": "82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "url": "/v2/monitor_instances/82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "created_at": "2024-07-02T07:23:45.509000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "drift_threshold": 0.05,
      "min_samples": 10,
      "train_drift_model": false
    },
    "thresholds": [
      {
        "metric_id": "drift_magnitude",
        "type": "upper_limit",
        "value": 0.5
      },
      {
        "metric_id": "predicted_accuracy",
        "type": "upper_limit",
        "value": 0.8
      },
      {
        "metric_id": "data_dr

### Check monitor instance status

In [37]:
drift_status = None

while drift_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
    drift_status = monitor_instance_details.entity.status.state
    if drift_status not in ("active", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_status)
        time.sleep(30)

print(datetime.now().strftime("%H:%M:%S"), drift_status)

07:26:03 preparing


07:26:33 preparing


07:27:03 preparing


07:27:33 preparing


07:28:03 active


### Run an on-demand evaluation

In [38]:
# Check Drift monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=drift_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "url": "/v2/monitor_instances/82cdf0a0-3a6e-4302-ac00-4061b46411e5",
    "created_at": "2024-07-02T07:23:45.509000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-02T07:28:02.598000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "drift",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0

In [39]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "ad470763-0d9a-4dac-80d2-b008cce3e388",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:ad470763-0d9a-4dac-80d2-b008cce3e388",
    "url": "/v2/monitor_instances/82cdf0a0-3a6e-4302-ac00-4061b46411e5/runs/ad470763-0d9a-4dac-80d2-b008cce3e388",
    "created_at": "2024-07-02T07:51:11.003000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_status": {
        "model_name": null,
        "state": "finished"
      },
      "data_drift_enabled": true,
      "data_drift_threshold": 0.1,
      "drift_buffer_range": [
        -4.5,
        4.5
      ],
      "drift_model_version": null,
      "drift_threshold": 0.05,
      "last_drift_run_end_timestamp": "2024-07-02T07:49:44.414541Z",
      "last_drift_run_start_timestamp": "2024-07-02T07:47:40.900330Z",
      "last_record_window_end_timestamp": "2024-07-02T07:47:42.298212Z",
      "last_record_window_start_timestamp": "2024-07-02T07

In [40]:
# Check run status

drift_run_status = None
while drift_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=drift_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    drift_run_status = monitoring_run_details.entity.status.state
    if drift_run_status not in ("finished", "error"):
        print(datetime.now().strftime("%H:%M:%S"), drift_run_status)
        time.sleep(30)
        
print(datetime.now().strftime("%H:%M:%S"), drift_run_status)

07:51:11 running
07:51:41 running
07:52:11 running
07:52:41 running
07:53:11 running
07:53:41 finished


### Display drift metrics

In [41]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-07-02 07:51:12.797976+00:00,data_drift_magnitude,2054c51e-55c8-4beb-9755-28a71bec3166,0.0,None,0.1,[],drift,82cdf0a0-3a6e-4302-ac00-4061b46411e5,ad470763-0d9a-4dac-80d2-b008cce3e388,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:51:12.797976+00:00,drift_magnitude,2054c51e-55c8-4beb-9755-28a71bec3166,0.0,None,0.05,[],drift,82cdf0a0-3a6e-4302-ac00-4061b46411e5,ad470763-0d9a-4dac-80d2-b008cce3e388,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 07:51:12.797976+00:00,predicted_accuracy,2054c51e-55c8-4beb-9755-28a71bec3166,0.8916666666666666,None,None,[],drift,82cdf0a0-3a6e-4302-ac00-4061b46411e5,ad470763-0d9a-4dac-80d2-b008cce3e388,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be


# 6. Fairness monitoring <a name="fairness"></a>

### Enable the fairness monitor

The following code cell, will enable the fairness monitor.

In [42]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = configuration_json["fairness_configuration"]["parameters"]
thresholds = configuration_json["fairness_configuration"]["thresholds"]

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result

fairness_monitor_instance_id = fairness_monitor_details.metadata.id
print(fairness_monitor_details)

{
  "metadata": {
    "id": "65d82bd1-e3c9-4c03-b62c-1440013e8c84",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:65d82bd1-e3c9-4c03-b62c-1440013e8c84",
    "url": "/v2/monitor_instances/65d82bd1-e3c9-4c03-b62c-1440013e8c84",
    "created_at": "2024-07-02T07:54:11.487000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "fairness",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "training_data_last_processed_time": "2024-06-30T11:07:29.827790Z",
      "features": [
        {
          "minority": [
            "female"
          ],
          "feature": "Sex",
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "majority": [
            "male"
          ],
          "t

### Check monitor instance status

In [43]:
fairness_state = fairness_monitor_details.entity.status.state

while fairness_state not in ("active", "error"):
    print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=fairness_monitor_instance_id).result
    fairness_state = monitor_instance_details.entity.status.state
    time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), fairness_state)

07:54:47 preparing
07:55:17 active


### Run an on-demand evaluation

In [44]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id
print(monitoring_run_details)

{
  "metadata": {
    "id": "d4a6383a-4a71-4b8e-a844-cdc5f42345d3",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:d4a6383a-4a71-4b8e-a844-cdc5f42345d3",
    "url": "/v2/monitor_instances/65d82bd1-e3c9-4c03-b62c-1440013e8c84/runs/d4a6383a-4a71-4b8e-a844-cdc5f42345d3",
    "created_at": "2024-07-02T07:55:26.582000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "favourable_class": [
        "No Risk"
      ],
      "features": [
        {
          "feature": "Sex",
          "majority": [
            "male"
          ],
          "metric_ids": [
            "statistical_parity_difference",
            "fairness_value"
          ],
          "minority": [
            "female"
          ],
          "threshold": 0.95
        },
        {
          "feature": "Age",
          "majority": [
            [
              26,
              75
            ]
          ],
          "metric_ids": [
            "stati

In [45]:
# Check run status

fairness_run_status = monitoring_run_details.entity.status.state
while fairness_run_status not in ("finished", "error"):
    print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=fairness_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    fairness_run_status = monitoring_run_details.entity.status.state
    time.sleep(30)
        
print(datetime.utcnow().strftime("%H:%M:%S"), fairness_run_status)

07:55:28 running
07:55:58 running
07:56:28 running
07:56:58 running
07:57:29 running
07:57:59 running
07:58:29 running
07:58:59 running
07:59:29 running
07:59:59 running
08:00:29 running
08:00:59 running
08:01:30 finished


### Display fairness metrics

In [46]:
wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-07-02 08:00:29.847869+00:00,fairness_value,e35a40e5-d701-4f3e-ae60-eaf140bc7905,175.00000000000003,95.0,125.0,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,65d82bd1-e3c9-4c03-b62c-1440013e8c84,d4a6383a-4a71-4b8e-a844-cdc5f42345d3,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 08:00:29.847869+00:00,statistical_parity_difference,e35a40e5-d701-4f3e-ae60-eaf140bc7905,0.429,-0.15,0.15,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,65d82bd1-e3c9-4c03-b62c-1440013e8c84,d4a6383a-4a71-4b8e-a844-cdc5f42345d3,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 08:00:29.847869+00:00,fairness_value,e35a40e5-d701-4f3e-ae60-eaf140bc7905,142.85714285714286,95.0,125.0,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,65d82bd1-e3c9-4c03-b62c-1440013e8c84,d4a6383a-4a71-4b8e-a844-cdc5f42345d3,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be
2024-07-02 08:00:29.847869+00:00,statistical_parity_difference,e35a40e5-d701-4f3e-ae60-eaf140bc7905,0.3,-0.15,0.15,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,65d82bd1-e3c9-4c03-b62c-1440013e8c84,d4a6383a-4a71-4b8e-a844-cdc5f42345d3,subscription,e34b9b87-b6e1-4c53-b92e-cb80dea042be


# 7. Explainability monitoring <a name="explainability"></a>

### Enable the explainability monitor

#### Upload explainability configuration archive
In the following code cell, type the path to the explainability configuration archive tar ball.

In [47]:
with open("/path/to/dir/containing/explainability.tar.gz", mode="rb") as explainability_tar:
    wos_client.monitor_instances.upload_explainability_archive(subscription_id=subscription_id, archive=explainability_tar)

print("Uploaded explainability configuration archive successfully.")

Uploaded explainability configuration archive successfully.


In [48]:
import time

target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)

parameters = {
    # Comment the below lines to disable lime global explanation. 
    # Lime global explanation is available from Cloud Pak for Data version 4.6.4 onwards.
    "global_explanation": {
        "enabled": True,  # Flag to enable global explanation 
        "explanation_method": "lime",
        # "sample_size": 1000, # [Optional] The sample size of records to be used for generating payload data global explanation. If not specified entire data in the payload window is used.
    }
}

explainability_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_instance_id = explainability_monitor_details.metadata.id
print(explainability_monitor_details)

{
  "metadata": {
    "id": "0544bb00-bab6-4eae-8c2f-06c0e4796eec",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:0544bb00-bab6-4eae-8c2f-06c0e4796eec",
    "url": "/v2/monitor_instances/0544bb00-bab6-4eae-8c2f-06c0e4796eec",
    "created_at": "2024-07-02T08:01:45.637000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "global_explanation": {
        "enabled": true,
        "explanation_method": "lime"
      }
    },
    "thresholds": [
      {
        "metric_id": "global_explanation_stability",
        "type": "lower_limit",
        "value": 0.8
      }
    ],
    "schedule": {
      "repeat_interval": 1,
      "repeat_unit": "week",
      "start_time": {
        "type

### Check monitor instance status

In [49]:
explainability_status = None

while explainability_status not in ("active", "error"):
    monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
    explainability_status = monitor_instance_details.entity.status.state
    if explainability_status not in ("active", "error"):
        print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)
        time.sleep(30)

print(datetime.utcnow().strftime('%H:%M:%S'), explainability_status)

08:23:35 preparing
08:24:05 preparing
08:24:36 preparing
08:25:06 preparing
08:25:36 preparing
08:26:06 preparing
08:26:36 active


### Run an on-demand evaluation

In [50]:
# Check Explainbility monitor instance details

monitor_instance_details = wos_client.monitor_instances.get(monitor_instance_id=explainability_monitor_instance_id).result
print(monitor_instance_details)

{
  "metadata": {
    "id": "7c8c6942-0607-4849-9450-4209ea5b7adc",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:7c8c6942-0607-4849-9450-4209ea5b7adc",
    "url": "/v2/monitor_instances/7c8c6942-0607-4849-9450-4209ea5b7adc",
    "created_at": "2024-07-02T08:06:08.878000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-07-02T08:26:26.023000Z",
    "modified_by": "internal-service"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "explainability",
    "target": {
      "target_type": "subscription",
      "target_id": "e34b9b87-b6e1-4c53-b92e-cb80dea042be"
    },
    "parameters": {
      "config_modified_at": "2024-07-02T08:26:25.895753Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        

In [51]:
# Trigger on-demand run

monitoring_run_details = wos_client.monitor_instances.run(monitor_instance_id=explainability_monitor_instance_id).result
monitoring_run_id=monitoring_run_details.metadata.id

print(monitoring_run_details)

{
  "metadata": {
    "id": "6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:run:6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "url": "/v2/monitor_instances/7c8c6942-0607-4849-9450-4209ea5b7adc/runs/6102d8a3-3d07-4918-9c0f-6b6047a209c5",
    "created_at": "2024-07-02T08:26:49.232000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "parameters": {
      "config_modified_at": "2024-07-02T08:26:25.895753Z",
      "config_package_file": "explainability.tar.gz",
      "controllable_features": [],
      "explanations_count": {
        "failed": 0,
        "total": 0
      },
      "lime": {
        "enabled": true,
        "features_count": 10,
        "perturbations_count": 208
      },
      "validate_table_job_app_id": null,
      "validate_table_job_id": "2db53ca8-dc6c-476f-baf3-48e1998ce19d",
      "validate_table_job_output_path": "explainability_configuration/d0cf6915-6897-4afb-a08c-36f1af1c186d/

In [52]:
# Check run status

explainability_run_status = None
while explainability_run_status not in ("finished", "error"):
    monitoring_run_details = wos_client.monitor_instances.get_run_details(monitor_instance_id=explainability_monitor_instance_id, monitoring_run_id=monitoring_run_id).result
    explainability_run_status = monitoring_run_details.entity.status.state
    if explainability_run_status not in ("finished", "error"):
        print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)
        time.sleep(60)
        
print(datetime.utcnow().strftime("%H:%M:%S"), explainability_run_status)

08:26:56 running
08:27:56 running
08:28:56 running
08:29:56 running
08:30:56 running
08:31:57 running
08:32:57 finished


In [53]:
# View the global explanation stability metric. When lime global explanation is enabled, the monitor run computes global explanation and publishes global_explanation_stability metric.
wos_client.monitor_instances.show_metrics(monitor_instance_id=explainability_monitor_instance_id)

### Display sample explanations

In [54]:
explanations = wos_client.monitor_instances.get_all_explaination_tasks(subscription_id=subscription_id).result
print(explanations)

{
  "total_count": 12,
  "limit": 50,
  "offset": 0,
  "explanation_fields": [
    "explanation_task_id",
    "scoring_id",
    "created_at",
    "finished_at",
    "status",
    "prediction",
    "subscription_id",
    "deployment_id",
    "asset_name",
    "deployment_name",
    "probability",
    "explanation_type"
  ],
  "explanation_values": [
    [
      "4d418585-4b18-4547-a33c-15ab7ed49ae9",
      "sc1",
      "2024-07-02T08:28:10.782148Z",
      "2024-07-02T08:28:10.782510Z",
      "finished",
      "No Risk",
      "e34b9b87-b6e1-4c53-b92e-cb80dea042be",
      "78a0af9e-1014-4fb1-b22a-5e11f4fd70e7",
      "WML_IAE4",
      "WML_IAE4",
      0.8429199,
      "lime"
    ],
    [
      "4ccdb32b-c7d4-420d-9fdb-e31dcc7db204",
      "sc2",
      "2024-07-02T08:28:10.788687Z",
      "2024-07-02T08:28:10.788949Z",
      "finished",
      "No Risk",
      "e34b9b87-b6e1-4c53-b92e-cb80dea042be",
      "78a0af9e-1014-4fb1-b22a-5e11f4fd70e7",
      "WML_IAE4",
      "WML_IAE4",
      0.

## Congratulations!

You have finished the Batch demo for IBM Watson OpenScale by using Apache Spark on Cloud Pak for Data IBM Analytics Engine. You can now view the [Watson OpenScale Dashboard](https://url-to-your-cp4d-cluster/aiopenscale). Click the tile for the **German Credit model** to see the quality, drift and fairness monitors. Click the timeseries graph to get detailed information on transactions during a specific time window.